# Расчет цены на автомобиль
### Книга 03 - разработка прототипа модели для оценки автомобилей.

Постановка задачи:
Необходимо разработать модель, которая бы рассчитывала цену на выставленный на продажу автомобиль.  По условиям учебной задачи обучающих данных в явном виде не предоставлено. Только тестовые, собранные на авто-ру больше года назад. Необходимо самостоятельно разработать программу, которая бы собирала данные по объявлениям на том же сайте авто.ру. Дополнительная сложность - количество данных. Оцениваться работа будет по порядка 35к записей. Необходимо собрать порядка 140 тыс записей.  На самом сайте автору сейчас актуально порядка 90к объявлений.

С сайта авто.ру загружены два набора исходных данных:
- набор предложений по 12 брендам автомобилей (как в тестовом наборе), размером порядка 42к записей;
- набор предложений по 36 брендам автомобилей (как в данных от авторов задачи), размером порядка 91к записей.

Ближайший план работы:
- Перенести предлагаемое авторами решение baseline и сделать на его основе MVP. Для MVP взять данные из набора по 12 брендам, как и предлагают авторы.
- Обработку данных сделать в виде трансформера. Взять данные по 36 брендам и сравнить качество модели при увеличении набора данных.
- ... ?

 ### Построение MVP

За основу взято baseline решение от авторов условий. Задача - повторить результат.

In [41]:
import numpy as np
import pandas as pd
from ast import literal_eval

import re
import sys
import itertools
import datetime
from tqdm.notebook import tqdm
import pandas_profiling

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer

In [2]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
# !pip freeze > requirements.txt
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true))

In [105]:
train =  pd.read_csv('input/211121-12brands.csv')
test = pd.read_csv('input/test.csv')


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14,15,19,20,22,28,33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [106]:
train.head()

,Unnamed: 0,index,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,0,0,Внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x5/11057182...,040001,{'id': '0'},Автомобиль приобретался новым и с тех пор нахо...,3.0 LTR,249 N12,...,AUTOMATIC,EUROPEAN,2,"{'year': 2017, 'month': 5}",ORIGINAL,ALL_WHEEL_DRIVE,LEFT,True,True,4350000
1,1,1,Седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/1106015...,040001,"{'id': '9361242', 'name': '316i SE', 'availabl...",Юридически чистый и технически обслуженный авт...,1.6 LTR,136 N12,...,AUTOMATIC,EUROPEAN,3,"{'year': 2020, 'month': 5}",ORIGINAL,REAR_DRIVE,LEFT,True,True,1420000
2,2,2,Седан,BMW,https://auto.ru/cars/used/sale/bmw/m5/11057329...,CACECB,{'id': '0'},Продажа указанного БМВ осуществляется в формат...,4.4 LTR,600 N12,...,ROBOT,EUROPEAN,1,"{'year': 2014, 'month': 12}",ORIGINAL,REAR_DRIVE,LEFT,True,True,80000000
3,3,3,Купе,BMW,https://auto.ru/cars/used/sale/bmw/3er/1103184...,EE1D19,{'id': '0'},Бмв 316 по документам 1.8\nПо факту был устано...,1.8 LTR,90 N12,...,MECHANICAL,EUROPEAN,4,"{'year': 2017, 'month': 7}",DUPLICATE,REAR_DRIVE,LEFT,True,True,990000
4,4,4,Седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1105925...,040001,{'id': '0'},"Автомобиль в достойном состоянии. Чистый, не к...",4.8 LTR,367 N12,...,AUTOMATIC,EUROPEAN,2,"{'year': 2019, 'month': 1}",ORIGINAL,REAR_DRIVE,LEFT,True,True,645000


In [107]:
test.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


Удалим два "производственных" столбца, возникших при слиянии наборов.

In [108]:
train.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)

Проверим соответствие по столбцам с тестовым набором.

In [109]:
set(test.columns).difference(train.columns)

set()

Цель соответствия по столбцам с тестовым набором достигнута. )
Удалим и переименуем столбцы.

In [110]:
train.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'location',
       'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate',
       'model_info', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime',
       'priceCurrency', 'productionDate', 'sell_id', 'super_gen',
       'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы',
       'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня', 'price'],
      dtype='object')

In [111]:
train.drop(['image', 'parsing_unixtime'], axis=1, inplace=True)


In [112]:
test.drop(['image', 'parsing_unixtime'], axis=1, inplace=True)

Переименую столбцы в соответствии с моми представлением о прекрасном.

In [113]:
rename_dict= {'bodyType':'body', 'car_url': 'url', 'engineDisplacement':'displacement', 'enginePower':'power',  'fuelType':'fuel', 'modelDate':'model_year', 'model_name':'model', 'numberOfDoors': 'doors', 'priceCurrency':'curr', 'productionDate':'manuf_year', 'vehicleConfiguration' : 'configuration', 'vehicleTransmission': 'transmission', 'Владельцы':'num_owners', 'Владение':'ownership', 'ПТС':'car_license', 'Привод':'gear', 'Руль':'steering', 'Состояние':'condition', 'Таможня':'duties'}
train.rename( columns= rename_dict, inplace=True)
test.rename(columns=rename_dict, inplace=True)

In [114]:
train.head(20)

,body,brand,url,color,complectation_dict,description,displacement,power,location,equipment_dict,...,transmission,vendor,num_owners,ownership,car_license,gear,steering,condition,duties,price
0,Внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x5/11057182...,040001,{'id': '0'},Автомобиль приобретался новым и с тех пор нахо...,3.0 LTR,249 N12,Москва,"{'tinted-glass': True, 'esp': True, 'adaptive-...",...,AUTOMATIC,EUROPEAN,2,"{'year': 2017, 'month': 5}",ORIGINAL,ALL_WHEEL_DRIVE,LEFT,True,True,4350000
1,Седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/1106015...,040001,"{'id': '9361242', 'name': '316i SE', 'availabl...",Юридически чистый и технически обслуженный авт...,1.6 LTR,136 N12,Щелково,"{'tinted-glass': True, 'esp': True, 'usb': Tru...",...,AUTOMATIC,EUROPEAN,3,"{'year': 2020, 'month': 5}",ORIGINAL,REAR_DRIVE,LEFT,True,True,1420000
2,Седан,BMW,https://auto.ru/cars/used/sale/bmw/m5/11057329...,CACECB,{'id': '0'},Продажа указанного БМВ осуществляется в формат...,4.4 LTR,600 N12,Москва,"{'cruise-control': True, 'roller-blind-for-rea...",...,ROBOT,EUROPEAN,1,"{'year': 2014, 'month': 12}",ORIGINAL,REAR_DRIVE,LEFT,True,True,80000000
3,Купе,BMW,https://auto.ru/cars/used/sale/bmw/3er/1103184...,EE1D19,{'id': '0'},Бмв 316 по документам 1.8\nПо факту был устано...,1.8 LTR,90 N12,Москва,"{'engine-proof': True, 'tinted-glass': True, '...",...,MECHANICAL,EUROPEAN,4,"{'year': 2017, 'month': 7}",DUPLICATE,REAR_DRIVE,LEFT,True,True,990000
4,Седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1105925...,040001,{'id': '0'},"Автомобиль в достойном состоянии. Чистый, не к...",4.8 LTR,367 N12,Москва,"{'cruise-control': True, 'asr': True, 'tinted-...",...,AUTOMATIC,EUROPEAN,2,"{'year': 2019, 'month': 1}",ORIGINAL,REAR_DRIVE,LEFT,True,True,645000
5,Внедорожник 5 дв.,BMW,https://auto.ru/cars/new/sale/bmw/x5/110595336...,4A2197,"{'id': '21579251', 'name': 'M50d M Special', '...",Комплектация: M50d M Special При покупке автом...,3.0 LTR,400 N12,Москва,"{'sport-seats': True, 'multi-wheel': True, 'au...",...,AUTOMATIC,EUROPEAN,NaN,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,True,True,12390000
6,Купе,BMW,https://auto.ru/cars/used/sale/bmw/m4/11056365...,FFD600,"{'id': '22523789', 'name': 'Competition', 'ava...","Доброго времени суток, убедительная просьба, п...",3.0 LTR,510 N12,Москва,"{'cruise-control': True, 'asr': True, 'tinted-...",...,AUTOMATIC,EUROPEAN,1,"{'year': 2021, 'month': 8}",ORIGINAL,REAR_DRIVE,LEFT,True,True,8690000
7,Купе,BMW,https://auto.ru/cars/used/sale/bmw/6er/1105788...,0000CC,"{'id': '20510163', 'name': '640d xDrive', 'ava...",Автомобиль был куплен новым у официального дил...,3.0 LTR,313 N12,Москва,"{'cruise-control': True, 'start-stop-function'...",...,AUTOMATIC,EUROPEAN,4,"{'year': 2013, 'month': 11}",DUPLICATE,ALL_WHEEL_DRIVE,LEFT,True,True,3290000
8,Внедорожник 5 дв.,BMW,https://auto.ru/cars/new/sale/bmw/x5/110576161...,97948F,"{'id': '21642384', 'name': 'xDrive30d M Sport ...",Комплектация: xDrive30d M Sport Plus При покуп...,3.0 LTR,249 N12,Москва,"{'cruise-control': True, 'asr': True, 'tinted-...",...,AUTOMATIC,EUROPEAN,NaN,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,True,True,8950000
9,Внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x7/11057179...,97948F,"{'id': '21738418', 'name': 'xDrive30d M Sport ...",Автомобиль был куплен новым у официального дил...,3.0 LTR,249 N12,Москва,"{'asr': True, 'esp': True, 'adaptive-light': T...",...,AUTOMATIC,EUROPEAN,2,"{'year': 2020, 'month': 7}",ORIGINAL,ALL_WHEEL_DRIVE,LEFT,True,True,7790000


In [115]:
train.isna().sum()

body                      0
brand                     0
url                       0
color                     0
complectation_dict        0
description             872
displacement              0
power                     0
location                  0
equipment_dict            0
fuel                      0
mileage                   0
model_year                0
model_info                0
model                     0
name                      0
doors                     0
curr                      0
manuf_year                0
sell_id                   0
super_gen                 0
configuration             0
transmission              0
vendor                    0
num_owners            12304
ownership             33786
car_license             229
gear                      0
steering                  0
condition                 0
duties                    0
price                     0
dtype: int64

In [116]:
test.isna().sum()

body                      0
brand                     0
url                       0
color                     0
complectation_dict    28268
description               0
displacement              0
power                     0
equipment_dict         9996
fuel                      0
mileage                   0
model_year                0
model_info                0
model                     0
name                      0
doors                     0
curr                      0
manuf_year                0
sell_id                   0
super_gen                 0
configuration             0
transmission              0
vendor                    0
num_owners                0
ownership             22691
car_license               1
gear                      0
steering                  0
condition                 0
duties                    0
dtype: int64

В данных есть пропуски, с которым набо разбираться.
Структура пропусков сильно отличается - в тестовом наборе практически нет данных в словаре комплектации и очень много пропусков в словаре оборудования.  Восстановить не получится. Вероятно от этих столбцов стоит отказаться.

Сгенерируем в отдельный файл отчет об учебном наборе данных.

In [117]:
# train_profile =  train.profile_report()
# train_profile.to_file("12 Brans Train Review.html")

Отчет сохранен в отдельном файле.
**Выводы по отчету**: Набор данных состоит из 30 признаков и 42к объектов; Процент пропущенных значений - 3.5% (сделаем позже). В наборе данных нет дубликатов. В наборе есть 24 категориальных столбца и 8 столбцов имеют неподдерживаемый тип данных. Существует сильная корреляция между некоторыми столбцами. Существует много столбцов с высокой кардинальностью (разнообразием данных).

### Обработка столбцов и превращение в признаки

In [118]:
train.body = train.body.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
train.body.value_counts(normalize=True)

внедорожник      0.514508
седан            0.270656
лифтбек          0.078485
хэтчбек          0.048923
универсал        0.026780
минивэн          0.021025
купе             0.017766
компактвэн       0.007967
пикап            0.006469
кабриолет        0.002331
фургон           0.001641
родстер          0.001427
купе-хардтоп     0.001403
микровэн         0.000309
седан-хардтоп    0.000214
лимузин          0.000071
bodytype         0.000024
Name: body, dtype: float64

In [119]:
test.body = test.body.apply(lambda x:x.lower().split()[0].strip() if isinstance(x,str) else x)
test.body.value_counts(normalize=True)

седан            0.379202
внедорожник      0.355936
хэтчбек          0.075218
лифтбек          0.064349
универсал        0.036787
минивэн          0.028686
купе             0.023756
компактвэн       0.014098
пикап            0.008361
купе-хардтоп     0.004555
родстер          0.002854
фургон           0.002797
кабриолет        0.002595
седан-хардтоп    0.000346
микровэн         0.000202
лимузин          0.000173
тарга            0.000058
фастбек          0.000029
Name: body, dtype: float64

In [120]:
body_types=list(train.body.unique()); body_types

['внедорожник',
 'седан',
 'купе',
 'хэтчбек',
 'универсал',
 'родстер',
 'кабриолет',
 'компактвэн',
 'лифтбек',
 'bodytype',
 'купе-хардтоп',
 'минивэн',
 'пикап',
 'фургон',
 'микровэн',
 'лимузин',
 'седан-хардтоп']

In [121]:
train.loc[train.body=='bodytype'].head()

,body,brand,url,color,complectation_dict,description,displacement,power,location,equipment_dict,...,transmission,vendor,num_owners,ownership,car_license,gear,steering,condition,duties,price
1784,bodytype,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,location,equipment_dict,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price


Это явный пропуск в данных, возможно сбой программы сбора данных. Устраняю его.

In [122]:
train.drop(train.loc[train.body=='bodytype'].index, inplace=True)

In [123]:
train.loc[train.body=='bodytype'].head()

,body,brand,url,color,complectation_dict,description,displacement,power,location,equipment_dict,...,transmission,vendor,num_owners,ownership,car_license,gear,steering,condition,duties,price


18 типов кузовов - избыточная информация. Сожмем ее с помощью словаря.

In [124]:
body_dict={ 'кабриолет':'coupe', 'седан':'sedan', 'внедорожник':'SUV', 'купе':'coupe', 'родстер':'coupe',
            'хэтчбек':'f_back', 'фастбек':'f_back', 'лифтбек': 'f_back', 'универсал': 'wagon', 'пикап' : 'SUV', 'минивэн': 'MPV', 'компактвэн': 'MPV', 'купе-хардтоп': 'coupe', 'фургон': 'MPV', 'микровэн': 'MPV', 'тарга': 'coupe', 'лимузин': 'sedan', 'седан-хардтоп': 'sedan' }

In [125]:
train['body_type'] = train['body'].apply(lambda x: body_dict[x])
train.body_type.value_counts(normalize=True)

SUV       0.520989
sedan     0.270948
f_back    0.127411
MPV       0.030943
wagon     0.026781
coupe     0.022928
Name: body_type, dtype: float64

In [126]:
test['body_type'] = test['body'].apply(lambda x: body_dict[x])
test.body_type.value_counts(normalize=True)

sedan     0.379721
SUV       0.364297
f_back    0.139595
MPV       0.045782
wagon     0.036787
coupe     0.033818
Name: body_type, dtype: float64

В учебном и тестовом наборе несколько разное распределение по типам корпусов. Разные наборы данных.

In [127]:
train.brand.unique()

array(['BMW', 'Volkswagen', 'Nissan', 'Mercedes-Benz', 'Toyota', 'Audi',
       'Mitsubishi', 'Skoda', 'Volvo', 'Honda', 'Infiniti', 'Lexus'],
      dtype=object)

In [128]:
test.brand.unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

In [129]:
train.brand = train.brand.apply(lambda x: x.upper())

In [130]:
train.brand.value_counts(normalize=True)

MERCEDES-BENZ    0.134951
SKODA            0.103722
NISSAN           0.102533
BMW              0.100678
AUDI             0.097562
TOYOTA           0.095992
VOLKSWAGEN       0.095017
MITSUBISHI       0.093400
VOLVO            0.055369
LEXUS            0.047972
HONDA            0.043953
INFINITI         0.028850
Name: brand, dtype: float64

In [131]:
train.color.value_counts()

040001    12779
FAFBFB     8992
97948F     5934
0000CC     4055
CACECB     3436
200204     1897
EE1D19     1811
007F00      982
C49648      765
22A0F8      357
FF8649      265
660099      245
DEA522      186
FFD600      171
4A2197      162
FFC0CB        8
Name: color, dtype: int64

In [132]:
color_dict = {'040001': 'чёрный', 'FAFBFB': 'белый', '97948F': 'серый', 'CACECB': 'серебристый', '0000CC': 'синий', '200204': 'коричневый', 'EE1D19': 'красный',  '007F00': 'зелёный', 'C49648': 'бежевый', '22A0F8': 'голубой', '660099': 'пурпурный', 'DEA522': 'золотистый', '4A2197': 'фиолетовый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}

In [133]:
train.color.replace(to_replace=color_dict, inplace=True)
train.color.value_counts(normalize=True)

чёрный         0.303936
белый          0.213866
серый          0.141134
синий          0.096444
серебристый    0.081722
коричневый     0.045118
красный        0.043073
зелёный        0.023356
бежевый        0.018195
голубой        0.008491
оранжевый      0.006303
пурпурный      0.005827
золотистый     0.004424
жёлтый         0.004067
фиолетовый     0.003853
розовый        0.000190
Name: color, dtype: float64

In [134]:
test.color.value_counts(normalize=True)

чёрный         0.317419
белый          0.176815
серебристый    0.118578
серый          0.114859
синий          0.094822
красный        0.044139
коричневый     0.043274
зелёный        0.027446
бежевый        0.018422
голубой        0.013175
золотистый     0.007669
пурпурный      0.007323
фиолетовый     0.006314
жёлтый         0.005593
оранжевый      0.003777
розовый        0.000375
Name: color, dtype: float64

"Ваша машина может быть любого цвета, до тех пор пока этот цвет - черный!"

In [135]:
train.head()

,body,brand,url,color,complectation_dict,description,displacement,power,location,equipment_dict,...,vendor,num_owners,ownership,car_license,gear,steering,condition,duties,price,body_type
0,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x5/11057182...,чёрный,{'id': '0'},Автомобиль приобретался новым и с тех пор нахо...,3.0 LTR,249 N12,Москва,"{'tinted-glass': True, 'esp': True, 'adaptive-...",...,EUROPEAN,2,"{'year': 2017, 'month': 5}",ORIGINAL,ALL_WHEEL_DRIVE,LEFT,True,True,4350000,SUV
1,седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/1106015...,чёрный,"{'id': '9361242', 'name': '316i SE', 'availabl...",Юридически чистый и технически обслуженный авт...,1.6 LTR,136 N12,Щелково,"{'tinted-glass': True, 'esp': True, 'usb': Tru...",...,EUROPEAN,3,"{'year': 2020, 'month': 5}",ORIGINAL,REAR_DRIVE,LEFT,True,True,1420000,sedan
2,седан,BMW,https://auto.ru/cars/used/sale/bmw/m5/11057329...,серебристый,{'id': '0'},Продажа указанного БМВ осуществляется в формат...,4.4 LTR,600 N12,Москва,"{'cruise-control': True, 'roller-blind-for-rea...",...,EUROPEAN,1,"{'year': 2014, 'month': 12}",ORIGINAL,REAR_DRIVE,LEFT,True,True,80000000,sedan
3,купе,BMW,https://auto.ru/cars/used/sale/bmw/3er/1103184...,красный,{'id': '0'},Бмв 316 по документам 1.8\nПо факту был устано...,1.8 LTR,90 N12,Москва,"{'engine-proof': True, 'tinted-glass': True, '...",...,EUROPEAN,4,"{'year': 2017, 'month': 7}",DUPLICATE,REAR_DRIVE,LEFT,True,True,990000,coupe
4,седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1105925...,чёрный,{'id': '0'},"Автомобиль в достойном состоянии. Чистый, не к...",4.8 LTR,367 N12,Москва,"{'cruise-control': True, 'asr': True, 'tinted-...",...,EUROPEAN,2,"{'year': 2019, 'month': 1}",ORIGINAL,REAR_DRIVE,LEFT,True,True,645000,sedan


In [136]:
test.head()

,body,brand,url,color,complectation_dict,description,displacement,power,equipment_dict,fuel,...,transmission,vendor,num_owners,ownership,car_license,gear,steering,condition,duties,body_type
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,f_back
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,f_back
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,f_back
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,f_back
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,f_back


 Анализ текстовых признаков:
 complectation_dict - можно вытащить информацию из признака avaliable_options, если разобрать словарь в текстовом виде. Но в тестовом наборе много пропусков данных.  Столбец под удаление.
 description - с помощью NLP можно было бы что-то вытащить, но сложно. В удаление.
 location - можно в дальнейшем сделать словарик расстояния до города для корректировки цены. Но отсутствует в тестовом наборе. Под удаление.
 equipment_dict - можно вытащить информацию об опциях, если разобрать словарь в текстовом виде. Но в тестовом наборе много пропусков данных. Столбец под удаление.
 fuel - категории на разных языках.
 model_info - дублирующая информация. Под удаление.
 model - здесь была ошибка. Похоже устранена.
 name - дублирующее имя, в удаление
 curr - проверить что там всего один признак и удалить
 super_gen - описание автомобиля. Важная детальная информация, но состав полей отличается от текущего состояния к 2020 году. Ошибка сборки данных. Пока игнорирую. В дальнейшем может быть вернусь и сделаю сбор информации еще раз.
 configuration - короткое описание конфигурации дублируется - под удаление
 transmission - использованы разные языки для категорий
 vendor - проверить, вероятно унаследованный признак, аналог расположения руля.
 num_owners - разные типы информации в test и train. Объединить.
 ownership - много пустых строк, в игнор
 gear - сделать сокращения, разные языки в описании
 steering - разные языки
 condition - ????, разные языки
 duties - вероятно унаследованный признак, разные языки, убрать

In [137]:
def unique_keys(feat):
    lbl = set()
    for row in feat:
        lbl.update(set(list(row.keys())))
    return lbl

In [138]:
train['super_gen_dict'] = train.super_gen.apply(literal_eval)
unique_keys(train.super_gen_dict)

{'id', 'name', 'price_segment', 'ru_name', 'year_from', 'year_to'}

In [139]:
test['super_gen_dict'] = test.super_gen.apply(literal_eval)
unique_keys(test.super_gen_dict)

{'acceleration',
 'clearance_max',
 'clearance_min',
 'displacement',
 'engine_type',
 'fuel_rate',
 'gear_type',
 'human_name',
 'id',
 'name',
 'nameplate',
 'power',
 'power_kvt',
 'transmission'}

In [140]:
train['model_info_dict'] = train.model_info.apply(literal_eval)
unique_keys(train.model_info_dict)

{'code', 'morphology', 'name', 'nameplate', 'ru_name'}

In [143]:
train['complectation_dict_dict'] = train.complectation_dict.apply(literal_eval)
unique_keys(train.complectation_dict_dict)

{'available_options', 'id', 'name', 'vendor_colors'}

In [144]:
train['equipment_dict_dict'] = train.equipment_dict.apply(literal_eval)
unique_keys(train.equipment_dict_dict)

{'12-inch-wheels',
 '12v-socket',
 '13-inch-wheels',
 '14-inch-wheels',
 '15-inch-wheels',
 '16-inch-wheels',
 '17-inch-wheels',
 '18-inch-wheels',
 '19-inch-wheels',
 '20-inch-wheels',
 '21-inch-wheels',
 '218',
 '22-inch-wheels',
 '220v-socket',
 '228',
 '23-inch-wheels',
 '234',
 '239',
 '28-inch-wheels',
 '301',
 '313',
 '345',
 '357',
 '360-camera',
 '3FB',
 '3L5',
 '3S1',
 '401',
 '414',
 '41K',
 '440',
 '443',
 '4D3',
 '4X4',
 '500',
 '51U',
 '531',
 '540',
 '597',
 '608',
 '61U',
 '632',
 '6E3',
 '6NQ',
 '6XF',
 '7K1',
 '7X5',
 '810',
 '840',
 '872',
 '873',
 '881',
 '8RF',
 '8RM',
 '8T2',
 '8X1',
 '9JD',
 '9M9',
 '9R1',
 '9VS',
 'F48',
 'F62',
 'KA2',
 'N4M',
 'PCH',
 'PE6',
 'PX2',
 'U25',
 'abs',
 'activ-suspension',
 'adaptive-light',
 'adj-pedals',
 'air-suspension',
 'airbag-curtain',
 'airbag-driver',
 'airbag-passenger',
 'airbag-rear-side',
 'airbag-side',
 'airbrush',
 'alarm',
 'alcantara',
 'alloy-wheel-disks',
 'android-auto',
 'apple-carplay',
 'armored',
 'ashtra

In [145]:
test.drop(['super_gen_dict'], axis=1, inplace=True)
train.drop(['super_gen_dict', 'equipment_dict_dict', 'complectation_dict_dict', 'model_info_dict' ], axis=1, inplace=True)

In [34]:
cols2drop = ['description', 'name', 'complectation_dict', 'equipment_dict', 'model_info', 'curr', 'super_gen','configuration', 'condition', 'duties']
train.drop(cols2drop, axis=1, inplace=True)
test.drop(cols2drop, axis=1, inplace=True)

In [35]:
if 'location' in train.columns:
    train.drop(['location'], axis=1, inplace=True)

In [37]:
train.head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,super_gen,transmission,vendor,num_owners,ownership,car_license,gear,steering,price,body_type
0,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x5/11057182...,чёрный,3.0 LTR,249 N12,DIESEL,99869,2013,X5,...,"{'id': '10382710', 'name': 'III (F15)', 'ru_na...",AUTOMATIC,EUROPEAN,2,"{'year': 2017, 'month': 5}",ORIGINAL,ALL_WHEEL_DRIVE,LEFT,4350000,SUV
1,седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/1106015...,чёрный,1.6 LTR,136 N12,GASOLINE,129000,2011,3ER,...,"{'id': '7744658', 'name': 'VI (F3x)', 'ru_name...",AUTOMATIC,EUROPEAN,3,"{'year': 2020, 'month': 5}",ORIGINAL,REAR_DRIVE,LEFT,1420000,sedan
2,седан,BMW,https://auto.ru/cars/used/sale/bmw/m5/11057329...,серебристый,4.4 LTR,600 N12,GASOLINE,37000,2013,M5,...,"{'id': '22613592', 'name': 'V (F10) Рестайлинг...",ROBOT,EUROPEAN,1,"{'year': 2014, 'month': 12}",ORIGINAL,REAR_DRIVE,LEFT,80000000,sedan
3,купе,BMW,https://auto.ru/cars/used/sale/bmw/3er/1103184...,красный,1.8 LTR,90 N12,GASOLINE,440000,1982,3ER,...,"{'id': '3473269', 'name': 'II (E30)', 'ru_name...",MECHANICAL,EUROPEAN,4,"{'year': 2017, 'month': 7}",DUPLICATE,REAR_DRIVE,LEFT,990000,coupe
4,седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1105925...,чёрный,4.8 LTR,367 N12,GASOLINE,129500,2005,7ER,...,"{'id': '2305656', 'name': 'IV (E65/E66) Рестай...",AUTOMATIC,EUROPEAN,2,"{'year': 2019, 'month': 1}",ORIGINAL,REAR_DRIVE,LEFT,645000,sedan


In [38]:
test.head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,sell_id,super_gen,transmission,vendor,num_owners,ownership,car_license,gear,steering,body_type
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,1.2 LTR,105 N12,бензин,74000,2013,OCTAVIA,...,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,f_back
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,1.6 LTR,110 N12,бензин,60563,2017,OCTAVIA,...,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,f_back
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,1.8 LTR,152 N12,бензин,88000,2013,SUPERB,...,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,f_back
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,1.6 LTR,110 N12,бензин,95000,2013,OCTAVIA,...,1100937408,"{""id"":""20105521"",""displacement"":1598,""engine_t...",автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,f_back
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,1.8 LTR,152 N12,бензин,58536,2008,OCTAVIA,...,1101037972,"{""id"":""4561004"",""displacement"":1798,""engine_ty...",автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,f_back


In [47]:
test.super_gen_dict[0]

{'id': '10373605',
 'displacement': 1197,
 'engine_type': 'GASOLINE',
 'gear_type': 'FORWARD_CONTROL',
 'transmission': 'ROBOT',
 'power': 105,
 'power_kvt': 77,
 'human_name': '1.2 AMT (105 л.с.)',
 'acceleration': 10.5,
 'clearance_min': 155,
 'fuel_rate': 5}

In [39]:
d = list(train.displacement.unique())
d.sort(); d

['0.0 LTR',
 '0.6 LTR',
 '0.7 LTR',
 '1.0 LTR',
 '1.1 LTR',
 '1.2 LTR',
 '1.3 LTR',
 '1.4 LTR',
 '1.5 LTR',
 '1.6 LTR',
 '1.7 LTR',
 '1.8 LTR',
 '1.9 LTR',
 '2.0 LTR',
 '2.1 LTR',
 '2.2 LTR',
 '2.3 LTR',
 '2.4 LTR',
 '2.5 LTR',
 '2.6 LTR',
 '2.7 LTR',
 '2.8 LTR',
 '2.9 LTR',
 '3.0 LTR',
 '3.1 LTR',
 '3.2 LTR',
 '3.3 LTR',
 '3.4 LTR',
 '3.5 LTR',
 '3.6 LTR',
 '3.7 LTR',
 '3.8 LTR',
 '4.0 LTR',
 '4.1 LTR',
 '4.2 LTR',
 '4.3 LTR',
 '4.4 LTR',
 '4.5 LTR',
 '4.6 LTR',
 '4.7 LTR',
 '4.8 LTR',
 '4.9 LTR',
 '5.0 LTR',
 '5.2 LTR',
 '5.4 LTR',
 '5.5 LTR',
 '5.6 LTR',
 '5.7 LTR',
 '5.8 LTR',
 '5.9 LTR',
 '6.0 LTR',
 '6.2 LTR',
 '6.3 LTR',
 '6.6 LTR']

In [40]:
train.loc[train.displacement=='0.0 LTR'].head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,super_gen,transmission,vendor,num_owners,ownership,car_license,gear,steering,price,body_type
29,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11041725...,чёрный,0.0 LTR,170 N12,ELECTRO,45000,2017,I3,...,"{'id': '21080702', 'name': 'I (I01) Рестайлинг...",AUTOMATIC,EUROPEAN,1,"{'year': 2021, 'month': 5}",ORIGINAL,REAR_DRIVE,LEFT,2249998,f_back
626,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11060008...,синий,0.0 LTR,170 N12,ELECTRO,42000,2017,I3,...,"{'id': '21080702', 'name': 'I (I01) Рестайлинг...",AUTOMATIC,EUROPEAN,1,"{'year': 2021, 'month': 3}",ORIGINAL,REAR_DRIVE,LEFT,2390000,f_back
636,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11060133...,белый,0.0 LTR,170 N12,ELECTRO,16909,2017,I3,...,"{'id': '21080702', 'name': 'I (I01) Рестайлинг...",AUTOMATIC,EUROPEAN,4,NaN,ORIGINAL,REAR_DRIVE,LEFT,1999000,f_back
751,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11059998...,белый,0.0 LTR,184 N12,ELECTRO,34850,2017,I3,...,"{'id': '21080702', 'name': 'I (I01) Рестайлинг...",AUTOMATIC,EUROPEAN,1,"{'year': 2018, 'month': 8}",ORIGINAL,REAR_DRIVE,LEFT,2490000,f_back
763,хэтчбек,BMW,https://auto.ru/cars/used/sale/bmw/i3/11059985...,чёрный,0.0 LTR,184 N12,ELECTRO,27250,2017,I3,...,"{'id': '21080702', 'name': 'I (I01) Рестайлинг...",AUTOMATIC,EUROPEAN,1,"{'year': 2021, 'month': 9}",ORIGINAL,REAR_DRIVE,LEFT,2545000,f_back


Неожиданно наткнулся на автомобили с электродвигателем.

In [41]:
train.displacement = train.displacement.apply(lambda x: x.replace(" LTR", "0.0 LTR") if x == " LTR" else x)
train.displacement = train.displacement.apply(lambda x: float(x.replace("LTR", "")) if isinstance(x, str) else x)
test.displacement = test.displacement.apply(lambda x: x.replace(" LTR", "0.0 LTR") if x == " LTR" else x)
test.displacement = test.displacement.apply(lambda x: float(x.replace("LTR", "")) if isinstance(x, str) else x)


In [42]:
train.displacement.value_counts(normalize=True)

2.0    0.299584
1.6    0.124628
3.0    0.107076
1.4    0.067570
2.5    0.065359
1.8    0.050731
2.4    0.045142
3.5    0.037436
4.0    0.019336
1.9    0.017125
2.9    0.016268
1.5    0.014223
1.3    0.011916
2.8    0.009609
4.4    0.009514
4.5    0.009395
2.1    0.009204
1.2    0.008657
4.7    0.007278
5.7    0.005898
5.6    0.005851
2.7    0.005637
5.5    0.005256
3.7    0.005185
3.2    0.005018
4.6    0.004662
0.0    0.004281
2.3    0.003354
4.2    0.003044
3.6    0.002878
2.2    0.002688
5.0    0.001903
3.3    0.001831
3.4    0.001594
1.7    0.001189
3.8    0.001118
1.0    0.001070
0.7    0.000999
3.1    0.000975
4.8    0.000904
4.3    0.000714
6.2    0.000690
6.0    0.000690
5.4    0.000547
2.6    0.000523
0.6    0.000381
4.1    0.000262
5.9    0.000190
5.2    0.000143
6.6    0.000119
4.9    0.000119
6.3    0.000095
1.1    0.000071
5.8    0.000071
Name: displacement, dtype: float64

In [43]:
test.displacement.value_counts(normalize=True)

2.0    0.213083
1.6    0.143372
3.0    0.112870
1.8    0.096552
2.5    0.071412
2.4    0.050308
3.5    0.049299
1.4    0.047137
1.5    0.020902
1.2    0.013983
4.4    0.012887
2.8    0.011071
4.7    0.010869
1.3    0.010523
5.5    0.010408
4.0    0.010264
3.2    0.010033
4.5    0.009975
2.1    0.007813
3.7    0.006977
2.2    0.006804
4.2    0.006573
1.9    0.006371
2.3    0.005853
2.7    0.005449
3.6    0.005276
5.6    0.004901
5.0    0.004555
5.7    0.003892
2.9    0.003777
4.6    0.003114
3.1    0.002537
4.8    0.002422
1.7    0.001701
1.0    0.001643
3.8    0.001643
2.6    0.001586
0.0    0.001586
6.0    0.001528
0.7    0.001528
3.3    0.001269
3.4    0.001038
4.1    0.001009
4.3    0.000923
5.4    0.000894
6.2    0.000807
5.2    0.000346
5.9    0.000346
6.3    0.000231
5.8    0.000202
6.6    0.000173
4.9    0.000115
1.1    0.000115
3.9    0.000029
5.3    0.000029
Name: displacement, dtype: float64

 Самый популярный двигатель имеет объем 2 литра.

In [44]:
train.power.unique()

array(['249 N12', '136 N12', '600 N12', '90 N12', '367 N12', '400 N12',
       '510 N12', '313 N12', '625 N12', '265 N12', '102 N12', '192 N12',
       '320 N12', '170 N12', '258 N12', '231 N12', '544 N12', '555 N12',
       '125 N12', '184 N12', '143 N12', '264 N12', '105 N12', '86 N12',
       '347 N12', '575 N12', '286 N12', '218 N12', '381 N12', '272 N12',
       '407 N12', '243 N12', '150 N12', '306 N12', '507 N12', '190 N12',
       '560 N12', '115 N12', '118 N12', '129 N12', '431 N12', '113 N12',
       '450 N12', '245 N12', '340 N12', '238 N12', '326 N12', '177 N12',
       '235 N12', '355 N12', '530 N12', '360 N12', '193 N12', '448 N12',
       '163 N12', '156 N12', '140 N12', '420 N12', '95 N12', '269 N12',
       '100 N12', '46 N12', '204 N12', '116 N12', '333 N12', '211 N12',
       '410 N12', '75 N12', '370 N12', '445 N12', '462 N12', '609 N12',
       '343 N12', '300 N12', '610 N12', '139 N12', '321 N12', '315 N12',
       '387 N12', '197 N12', '174 N12', '110 N12', '224 

In [45]:
test.power.unique()

array(['105 N12', '110 N12', '152 N12', '200 N12', '102 N12', '150 N12',
       '90 N12', '180 N12', '220 N12', '122 N12', '70 N12', '140 N12',
       '125 N12', '54 N12', '86 N12', '75 N12', '64 N12', '95 N12',
       '260 N12', '170 N12', '80 N12', '68 N12', '160 N12', '115 N12',
       '280 N12', '53 N12', '60 N12', '143 N12', '42 N12', '101 N12',
       '58 N12', '193 N12', '79 N12', '30 N12', '100 N12', '50 N12',
       '163 N12', '225 N12', '420 N12', '211 N12', '245 N12', '560 N12',
       '500 N12', '249 N12', '450 N12', '605 N12', '250 N12', '354 N12',
       '120 N12', '290 N12', '230 N12', '350 N12', '204 N12', '255 N12',
       '340 N12', '177 N12', '272 N12', '372 N12', '210 N12', '130 N12',
       '300 N12', '190 N12', '239 N12', '435 N12', '333 N12', '271 N12',
       '326 N12', '238 N12', '310 N12', '233 N12', '252 N12', '133 N12',
       '460 N12', '520 N12', '400 N12', '525 N12', '367 N12', '265 N12',
       '550 N12', '580 N12', '88 N12', '165 N12', '430 N12', '335 N

In [46]:
train.power = train.power.apply(lambda x: float(x.replace("N12", "")) if isinstance(x, str) else x)
train.power.value_counts(normalize=True)

249.0    0.099251
150.0    0.092686
110.0    0.052563
144.0    0.036865
190.0    0.034606
           ...   
514.0    0.000024
316.0    0.000024
460.0    0.000024
522.0    0.000024
51.0     0.000024
Name: power, Length: 316, dtype: float64

In [47]:
test.power = test.power.apply(lambda x: float(x.replace("N12", "")) if isinstance(x, str) else x)
test.power.value_counts(normalize=True)

249.0    0.049242
150.0    0.048982
110.0    0.029954
170.0    0.028167
105.0    0.027533
           ...   
514.0    0.000029
626.0    0.000029
38.0     0.000029
32.0     0.000029
301.0    0.000029
Name: power, Length: 315, dtype: float64

In [48]:
train.fuel.unique()

array(['DIESEL', 'GASOLINE', 'HYBRID', 'ELECTRO', 'LPG'], dtype=object)

In [49]:
test.fuel.unique()

array(['бензин', 'дизель', 'гибрид', 'электро', 'газ'], dtype=object)

In [50]:
fuel_dict={'DIESEL':'D', 'дизель':'D', 'GASOLINE':'B', 'бензин':'B', 'HYBRID':'H', 'гибрид':'H', 'ELECTRO':'E', 'электро':'E', 'LPG':'G', 'газ':'G'}
train.fuel = train.fuel.apply(lambda x: fuel_dict[x])
test.fuel = test.fuel.apply(lambda x: fuel_dict[x])

In [51]:
train.fuel.value_counts(normalize=True)

B    0.792009
D    0.193697
H    0.009942
E    0.004281
G    0.000071
Name: fuel, dtype: float64

In [52]:
test.fuel.value_counts(normalize=True)

B    0.824569
D    0.167214
H    0.006429
E    0.001586
G    0.000202
Name: fuel, dtype: float64

In [53]:
train.mileage = train.mileage.apply(lambda x: float(x))
train.model_year = train.model_year.apply(lambda x: int(x))
train.manuf_year = train.manuf_year.apply(lambda x: int(x))
train.price = train.price.apply(lambda x: int(x))

test.mileage = test.mileage.apply(lambda x: float(x))
test.model_year = test.model_year.apply(lambda x: int(x))
test.manuf_year = test.manuf_year.apply(lambda x: int(x))
# В test нет колонки цены!!!  Учесть в трансформере!!!!

l = list(train.loc[train.brand=='BMW'].model_name.unique());l

In [54]:
train.doors.unique()

array(['5', '4', '2', '3', 5, 4, 2, 3], dtype=object)

In [55]:
test.doors.unique()

array([5, 4, 2, 3, 0])

In [173]:
test.loc[test.doors==0].head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,manuf_year,sell_id,transmission,vendor,num_owners,ownership,car_license,gear,steering,body_type
16944,кабриолет,MERCEDES,https://auto.ru/cars/used/sale/mercedes/simple...,белый,5.3,32.0,B,48000.0,1904,SIMPLEX,...,1904,1093802104,механическая,EUROPEAN,1 владелец,6 лет и 8 месяцев,Оригинал,задний,Правый,coupe


Это ошибка ввода данных. Надо заменить на 2

In [56]:
test.doors = test.doors.apply( lambda x: 2 if x ==0 else x)

In [57]:
train.doors = train.doors.apply(lambda x: int(x))
train.doors.value_counts(normalize=True)

5    0.676822
4    0.288929
2    0.023403
3    0.010846
Name: doors, dtype: float64

In [58]:
test.doors = test.doors.apply(lambda x: int(x))
test.doors.value_counts(normalize=True)

5    0.539209
4    0.407254
2    0.034798
3    0.018740
Name: doors, dtype: float64

In [59]:
train.num_owners.unique()

array(['2', '3', '1', '4', nan, 1.0, 2.0, 4.0, 3.0], dtype=object)

In [60]:
train.loc[train.num_owners.isna()].head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,sell_id,transmission,vendor,num_owners,ownership,car_license,gear,steering,price,body_type
5,внедорожник,BMW,https://auto.ru/cars/new/sale/bmw/x5/110595336...,фиолетовый,3.0,400.0,D,0.0,2018,X5,...,1105953369-4a4e8fb5,AUTOMATIC,EUROPEAN,NaN,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,12390000,SUV
8,внедорожник,BMW,https://auto.ru/cars/new/sale/bmw/x5/110576161...,серый,3.0,249.0,D,0.0,2018,X5,...,1105761617-1c273d38,AUTOMATIC,EUROPEAN,NaN,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,8950000,SUV
14,внедорожник,BMW,https://auto.ru/cars/new/sale/bmw/x5/110602585...,серый,3.0,249.0,D,0.0,2018,X5,...,1106025852-e958f17e,AUTOMATIC,EUROPEAN,NaN,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,9980000,SUV
19,внедорожник,BMW,https://auto.ru/cars/new/sale/bmw/x5/110593983...,серый,3.0,249.0,D,0.0,2018,X5,...,1105939838-73c137bb,AUTOMATIC,EUROPEAN,NaN,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,9980000,SUV
24,внедорожник,BMW,https://auto.ru/cars/new/sale/bmw/x5/110592839...,серый,3.0,400.0,D,0.0,2018,X5,...,1105928395-2644dc80,AUTOMATIC,EUROPEAN,NaN,NaN,ORIGINAL,ALL_WHEEL_DRIVE,LEFT,12188400,SUV


Судя по всему - у новых машин с нулевым пробегом число владельцев отсутствует. Заполню пропуски нулем. А остальное в целое. В соответствующем поле таблицы test используется иной формат. Приводим все к одному значению.

In [61]:
train.num_owners.fillna(0,inplace=True)
train.num_owners = train.num_owners.apply( lambda x: int(x[0]) if isinstance(x, str) else int(x))
train.num_owners = train.num_owners.apply(lambda x: 3 if x>2 else x)


train.num_owners.value_counts(normalize=True)

0    0.292639
3    0.288477
1    0.239339
2    0.179546
Name: num_owners, dtype: float64

In [62]:
test.num_owners.unique()

array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object)

In [63]:
test.loc[test.mileage<10.0].head(20)

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,manuf_year,sell_id,transmission,vendor,num_owners,ownership,car_license,gear,steering,body_type
5857,внедорожник,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1100...,белый,2.0,150.0,B,1.0,2016,CR_V,...,2019,1100623326,вариатор,JAPANESE,1 владелец,NaN,Оригинал,полный,Левый,SUV
7564,седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1098589...,чёрный,3.0,249.0,D,7.0,2019,7ER,...,2020,1098589736,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,sedan
7974,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x7/11011882...,чёрный,3.0,400.0,D,9.0,2018,X7,...,2020,1101188249,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,SUV
7975,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x6_m/110118...,серый,4.4,625.0,B,9.0,2019,X6_M,...,2020,1101188250,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,SUV
7993,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x4/11011882...,голубой,2.0,190.0,D,9.0,2018,X4,...,2020,1101188254,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,SUV
8037,седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1101188...,чёрный,3.0,249.0,D,9.0,2019,7ER,...,2020,1101188251,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,sedan
8291,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x5/10997851...,белый,3.0,249.0,D,6.0,2018,X5,...,2020,1099785186,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,SUV
9144,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x5/11007992...,белый,3.0,249.0,D,6.0,2018,X5,...,2020,1100799200,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,SUV
9398,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x7/11007986...,белый,3.0,249.0,D,6.0,2018,X7,...,2020,1100798608,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,SUV
10270,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x7/11003604...,чёрный,3.0,340.0,B,9.0,2018,X7,...,2020,1100360462,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,SUV


В тестовый набор добавлены придуманные данные. Не бывает не новых автомобилей с пробегом 1 км. Обычно разъезды по производственным территориям и т.д. у новой машины формируют показания типа 15-25 км. Необходимо учесть это при создании признака. Есть смысл заполнить километраж вместо нуля значением 10 и потом может быть взять натуральный логарифм.

Примечание:  На самом деле это оказалась ошибка восприятия данных. Я не учел, что по данным, собранным в 2020 году не могут быть представлены а/м выпуска 2021 года, все автомобили 2020 года - новые.  Позже я исправил эту ошибку.

In [64]:
test.num_owners = test.num_owners.apply( lambda x: int(x[0]) if isinstance(x, str) else int(x))
test.num_owners = test.num_owners.apply(lambda x: 3 if x>2 else x)
test.num_owners.value_counts(normalize=True)

3    0.460042
1    0.272704
2    0.267255
Name: num_owners, dtype: float64

In [65]:
train.mileage = train.mileage.apply(lambda x: 10.0 if x < 10.0 else x)
test.mileage = test.mileage.apply(lambda x: 10.0 if x < 10.0 else x)

В признаке ownership содержится информация о дате начала эксплуатации вероятно первым владельцем. Информация интересная, но очень много пропусков. Просто удалю ее.


In [66]:
train.drop(['ownership'], axis=1, inplace=True)
test.drop(['ownership'], axis=1, inplace=True)

In [67]:
test.transmission.unique()

array(['роботизированная', 'механическая', 'автоматическая', 'вариатор'],
      dtype=object)

In [68]:
train.transmission.unique()

array(['AUTOMATIC', 'ROBOT', 'MECHANICAL', 'VARIATOR'], dtype=object)

In [69]:
transmission_dict = {'AUTOMATIC':'AT', 'автоматическая':'AT', 'ROBOT': 'AMT', 'роботизированная':'AMT', 'MECHANICAL':'MT', 'механическая':'MT', 'VARIATOR':'CVT', 'вариатор':'CVT'}
test.transmission = test.transmission.apply(lambda x: transmission_dict[x])
train.transmission = train.transmission.apply(lambda x: transmission_dict[x])
test.transmission.value_counts(normalize=True)

AT     0.564954
MT     0.207836
CVT    0.115291
AMT    0.111918
Name: transmission, dtype: float64

In [70]:
train.transmission.value_counts(normalize=True)

AT     0.570674
CVT    0.154406
AMT    0.144631
MT     0.130289
Name: transmission, dtype: float64

In [71]:
test.car_license.unique()

array(['Оригинал', 'Дубликат', nan], dtype=object)

In [72]:
test.loc[test.car_license.isna()].head(10)

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,doors,manuf_year,sell_id,transmission,vendor,num_owners,car_license,gear,steering,body_type
10412,седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/3350400...,синий,2.5,170.0,B,276000.0,1998,3ER,4,1999,33504008,AT,EUROPEAN,3,NaN,задний,Левый,sedan


В данном случае это просто пропуск значения. Заполним по аналогии с train.

In [73]:
train.car_license.unique()

array(['ORIGINAL', 'DUPLICATE', nan], dtype=object)

In [74]:
license_dict={'ORIGINAL':'ORG', 'Оригинал':'ORG', 'DUPLICATE':'DPL', 'Дубликат':'DPL'}

In [75]:
train.loc[(train.num_owners==0)&train.car_license.isna()].head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,manuf_year,sell_id,transmission,vendor,num_owners,car_license,gear,steering,price,body_type
3397,седан,BMW,https://auto.ru/cars/new/sale/bmw/8er/11053123...,синий,4.4,530.0,B,10.0,2018,8ER,...,2021,1105312324-6da8535e,AT,EUROPEAN,0,NaN,ALL_WHEEL_DRIVE,LEFT,11770600,sedan
3940,лифтбек,VOLKSWAGEN,https://auto.ru/cars/new/sale/volkswagen/polo/...,белый,1.6,110.0,B,10.0,2020,POLO,...,2021,1105982265-b00cbcf5,AT,EUROPEAN,0,NaN,FORWARD_CONTROL,LEFT,1617196,f_back
3946,лифтбек,VOLKSWAGEN,https://auto.ru/cars/new/sale/volkswagen/polo/...,чёрный,1.6,110.0,B,10.0,2020,POLO,...,2021,1105956689-0c2d7fab,AT,EUROPEAN,0,NaN,FORWARD_CONTROL,LEFT,1799742,f_back
3960,лифтбек,VOLKSWAGEN,https://auto.ru/cars/new/sale/volkswagen/polo/...,серебристый,1.6,110.0,B,10.0,2020,POLO,...,2021,1105956428-d8af50d2,AT,EUROPEAN,0,NaN,FORWARD_CONTROL,LEFT,1631811,f_back
4963,внедорожник,VOLKSWAGEN,https://auto.ru/cars/new/sale/volkswagen/tigua...,синий,1.4,150.0,B,10.0,2020,TIGUAN,...,2021,1105956483-a208d986,AMT,EUROPEAN,0,NaN,FORWARD_CONTROL,LEFT,3031372,SUV


In [76]:
train.loc[(train.num_owners>0)&train.car_license.isna()].head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,manuf_year,sell_id,transmission,vendor,num_owners,car_license,gear,steering,price,body_type


In [77]:
train.loc[(train.car_license.isna())].head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,manuf_year,sell_id,transmission,vendor,num_owners,car_license,gear,steering,price,body_type
3397,седан,BMW,https://auto.ru/cars/new/sale/bmw/8er/11053123...,синий,4.4,530.0,B,10.0,2018,8ER,...,2021,1105312324-6da8535e,AT,EUROPEAN,0,NaN,ALL_WHEEL_DRIVE,LEFT,11770600,sedan
3940,лифтбек,VOLKSWAGEN,https://auto.ru/cars/new/sale/volkswagen/polo/...,белый,1.6,110.0,B,10.0,2020,POLO,...,2021,1105982265-b00cbcf5,AT,EUROPEAN,0,NaN,FORWARD_CONTROL,LEFT,1617196,f_back
3946,лифтбек,VOLKSWAGEN,https://auto.ru/cars/new/sale/volkswagen/polo/...,чёрный,1.6,110.0,B,10.0,2020,POLO,...,2021,1105956689-0c2d7fab,AT,EUROPEAN,0,NaN,FORWARD_CONTROL,LEFT,1799742,f_back
3960,лифтбек,VOLKSWAGEN,https://auto.ru/cars/new/sale/volkswagen/polo/...,серебристый,1.6,110.0,B,10.0,2020,POLO,...,2021,1105956428-d8af50d2,AT,EUROPEAN,0,NaN,FORWARD_CONTROL,LEFT,1631811,f_back
4963,внедорожник,VOLKSWAGEN,https://auto.ru/cars/new/sale/volkswagen/tigua...,синий,1.4,150.0,B,10.0,2020,TIGUAN,...,2021,1105956483-a208d986,AMT,EUROPEAN,0,NaN,FORWARD_CONTROL,LEFT,3031372,SUV


Оказалось, что при отсутствии владельца в графу ПТС может быть внесена ошибочная информация. Заполняю пропуски на ORIGINAL

In [78]:
train.car_license.fillna('ORIGINAL', inplace=True)
train.car_license = train.car_license.apply(lambda x: license_dict[x])
test.car_license.fillna('ORIGINAL', inplace=True)
test.car_license = test.car_license.apply(lambda x: license_dict[x])

test.car_license.value_counts(normalize=True)

ORG    0.867756
DPL    0.132244
Name: car_license, dtype: float64

In [79]:
train.car_license.value_counts(normalize=True)

ORG    0.906553
DPL    0.093447
Name: car_license, dtype: float64

In [80]:
train.gear.unique()

array(['ALL_WHEEL_DRIVE', 'REAR_DRIVE', 'FORWARD_CONTROL'], dtype=object)

In [81]:
test.gear.unique()

array(['передний', 'полный', 'задний'], dtype=object)

In [82]:
l=list(train.gear.unique()); l

['ALL_WHEEL_DRIVE', 'REAR_DRIVE', 'FORWARD_CONTROL']

In [83]:
gear_dict = {'REAR_DRIVE':'RWD', 'ALL_WHEEL_DRIVE':'AWD', 'FORWARD_CONTROL': 'FWD', 'задний':'RWD', 'полный':'AWD', 'передний': 'FWD'}
train.gear = train.gear.apply(lambda x: gear_dict[x])
test.gear = test.gear.apply(lambda x: gear_dict[x])
train.gear.value_counts(normalize=True)

AWD    0.547794
FWD    0.388560
RWD    0.063646
Name: gear, dtype: float64

In [84]:
test.gear.value_counts(normalize=True)

FWD    0.448740
AWD    0.433028
RWD    0.118232
Name: gear, dtype: float64

In [85]:
train.steering.unique()

array(['LEFT', 'RIGHT'], dtype=object)

In [86]:
test.steering.unique()

array(['Левый', 'Правый'], dtype=object)

In [87]:
steering_dict = {'LEFT':'L', 'RIGHT':'R', 'Левый':'L', 'Правый':'R'}
test.steering = test.steering.apply(lambda x: steering_dict[x])
train.steering = train.steering.apply(lambda x: steering_dict[x])
test.steering.value_counts(normalize=True)

L    0.959667
R    0.040333
Name: steering, dtype: float64

In [88]:
train.steering.value_counts(normalize=True)

L    0.974908
R    0.025092
Name: steering, dtype: float64

In [89]:
test.vendor.value_counts(normalize=True)

EUROPEAN    0.596264
JAPANESE    0.403736
Name: vendor, dtype: float64

In [90]:
train.vendor.value_counts(normalize=True)

EUROPEAN    0.587252
JAPANESE    0.412701
AMERICAN    0.000048
Name: vendor, dtype: float64

In [91]:
train.loc[train.vendor=='AMERICAN'].head()

,body,brand,url,color,displacement,power,fuel,mileage,model_year,model,...,manuf_year,sell_id,transmission,vendor,num_owners,car_license,gear,steering,price,body_type
5197,минивэн,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/euro...,зелёный,2.8,201.0,B,388500.0,1997,EUROVAN,...,2002,1105556887-b2652d4d,AT,AMERICAN,3,ORG,FWD,L,650000,MPV
6646,минивэн,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/euro...,синий,2.8,201.0,B,240082.0,1997,EUROVAN,...,2002,1105220954-e7371936,AT,AMERICAN,3,ORG,FWD,L,770000,MPV


В train и test представлены только европейские и японские автомобили. Фургоны модели Eurovan выпускались VW для американского рынка. Но в данном случае есть всего две записи с таким значением, что является выбросом. Откорректирую их на европейцев.

In [92]:
train.vendor = train.vendor.apply(lambda x: 'EUROPEAN' if x == 'AMERICAN' else x)
train.vendor.value_counts(normalize=True)

EUROPEAN    0.587299
JAPANESE    0.412701
Name: vendor, dtype: float64

In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42045 entries, 0 to 42045
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   body          42045 non-null  object 
 1   brand         42045 non-null  object 
 2   url           42045 non-null  object 
 3   color         42045 non-null  object 
 4   displacement  42045 non-null  float64
 5   power         42045 non-null  float64
 6   fuel          42045 non-null  object 
 7   mileage       42045 non-null  float64
 8   model_year    42045 non-null  int64  
 9   model         42045 non-null  object 
 10  doors         42045 non-null  int64  
 11  manuf_year    42045 non-null  int64  
 12  sell_id       42045 non-null  object 
 13  transmission  42045 non-null  object 
 14  vendor        42045 non-null  object 
 15  num_owners    42045 non-null  int64  
 16  car_license   42045 non-null  object 
 17  gear          42045 non-null  object 
 18  steering      42045 non-nu

После проведенных обработок еще раз посмотрю на исходные данные с помощью генератора отчетов.

In [94]:
# train_profile =  train.profile_report()
# train_profile.to_file("12 Brans Train Review Upd.html")

Выводы - ошибка исходных данных (не поддерживаемые типы данных) устранена.  В таблице много коррелирующих признаков.  Разделим все признаки по типам

In [95]:
train.columns

Index(['body', 'brand', 'url', 'color', 'displacement', 'power', 'fuel',
       'mileage', 'model_year', 'model', 'doors', 'manuf_year', 'sell_id',
       'transmission', 'vendor', 'num_owners', 'car_license', 'gear',
       'steering', 'price', 'body_type'],
      dtype='object')

In [96]:
target = ['price']
cat_cols = ['brand', 'color', 'fuel', 'model', 'doors', 'transmission', 'vendor', 'num_owners',  'car_license', 'gear', 'steering', 'body_type']
num_cols = [ 'displacement', 'power', 'mileage', 'model_year','manuf_year' ]
help_cols = ['sell_id','body','url', ]

Проверим корреляцию

In [97]:
plt.figure(figsize=(15, 8))
sns.heatmap(train[num_cols + target].corr().abs(), vmin=0, vmax=1, annot=True);

Здесь у меня не было графика - модули отказались "рисовать".

In [98]:
train[num_cols+target].corr()

,displacement,power,mileage,model_year,manuf_year,price
displacement,1.000000,0.752856,0.088105,-0.048290,-0.068342,0.321948
power,0.752856,1.000000,-0.170633,0.215469,0.189726,0.636815
mileage,0.088105,-0.170633,1.000000,-0.784805,-0.807301,-0.490443
model_year,-0.048290,0.215469,-0.784805,1.000000,0.979226,0.471123
manuf_year,-0.068342,0.189726,-0.807301,0.979226,1.000000,0.477412
price,0.321948,0.636815,-0.490443,0.471123,0.477412,1.000000


Похоже, что существенная корреляция между model_year и manuf_year будет мешать. Есть смысл в будущем сделать разницу между модельным годом и годом выпуска.

Примечание.  В данном случае была невыявлена еще одна ошибка. Если в сборе данных была допущена ошибка и в столбцах будут содержатся данные разных типов - происходит "магическое" воздействие на модули построения графики и они перестают работать.   Пришлось сначала убрать ошибку из исходных данных и пересохранить их, и тогда графики стали отображаться.

In [99]:
num_cols = [ 'displacement', 'power', 'mileage', 'manuf_year' ]

#### Кодирование категориальных переменных

In [100]:
for colum in cat_cols:
    train[colum] = train[colum].astype('category').cat.codes


In [101]:
X = train[num_cols+cat_cols]
y = train.price
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42045 entries, 0 to 42045
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   displacement  42045 non-null  float64
 1   power         42045 non-null  float64
 2   mileage       42045 non-null  float64
 3   manuf_year    42045 non-null  int64  
 4   brand         42045 non-null  int8   
 5   color         42045 non-null  int8   
 6   fuel          42045 non-null  int8   
 7   model         42045 non-null  int16  
 8   doors         42045 non-null  int8   
 9   transmission  42045 non-null  int8   
 10  vendor        42045 non-null  int8   
 11  num_owners    42045 non-null  int8   
 12  car_license   42045 non-null  int8   
 13  gear          42045 non-null  int8   
 14  steering      42045 non-null  int8   
 15  body_type     42045 non-null  int8   
dtypes: float64(3), int16(1), int64(1), int8(11)
memory usage: 2.1 MB


In [102]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, shuffle=True, random_state=42)

Обучение некоторых моделей

In [103]:
rfr = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1)
rfr.fit(X_train, y_train)
predict_rf = rfr.predict(X_test)

print(f"The MAPE mertics of the Random Forest model: {(mape(y_test, predict_rf) * 100):0.2f}%.")

rfr_log = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1)
rfr_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rfr_log.predict(X_test))

print(f"The MAPE mertic for the Random Forest model on ln(price) : {(mape(y_test, predict_rf_log) * 100):0.2f}%.")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


The MAPE mertics of the Random Forest model: 13.16%.


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s


The MAPE mertic for the Random Forest model on ln(price) : 11.58%.


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


Получен хороший промежуточный результат.  Метрика для RandomForestRegressor для случая натурального логарифмирования цены оказалась на уровне 11.58%  (В прошлой итерации у меня был признак location, который сейчас стерт, так как отсутствует в test и метрика была еще лучше - 11.26%)
Задача состоит в том, чтобы сделать функцию-трансформер для данных, применить ее к большому набору из 36 брендов и посмотреть на изменение итогового результата.

In [131]:
def initial_transformation(data):

    rename_dict= {'bodyType':'body', 'car_url': 'url', 'engineDisplacement':'displacement', 'enginePower':'power',  'fuelType':'fuel', 'modelDate':'model_year', 'model_name':'model', 'numberOfDoors': 'doors', 'priceCurrency':'curr', 'productionDate':'manuf_year', 'vehicleConfiguration' : 'configuration', 'vehicleTransmission': 'transmission', 'Владельцы':'num_owners', 'Владение':'ownership', 'ПТС':'car_license', 'Привод':'gear', 'Руль':'steering', 'Состояние':'condition', 'Таможня':'duties'}
    data.rename( columns= rename_dict, inplace=True)

    # print(data.columns)
    cols2drop = ['image', 'parsing_unixtime', 'description', 'name', 'complectation_dict', 'equipment_dict', 'model_info', 'curr', 'super_gen', 'configuration', 'condition', 'duties', 'ownership']
    data.drop(cols2drop, axis=1, inplace=True )
    if 'location' in data.columns:
        data.drop(['location'], axis=1, inplace=True)


    data.body = data.body.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
    bt = 'bodytype'; btl = list(data.body.unique())
    if bt in btl :
        data.drop(data.loc[data.body==bt].index, inplace=True)
    # print(list(data.body.unique()))
    body_dict = {'кабриолет': 'coupe', 'седан': 'sedan', 'внедорожник': 'SUV', 'купе': 'coupe', 'родстер': 'coupe',
                 'хэтчбек': 'f_back', 'лифтбек': 'f_back', 'универсал': 'wagon', 'пикап': 'SUV', 'минивэн': 'MPV',
                 'компактвэн': 'MPV', 'купе-хардтоп': 'coupe', 'фургон': 'MPV', 'микровэн': 'MPV', 'тарга': 'coupe', 'фастбек': 'f_back', 'лимузин': 'sedan', 'седан-хардтоп': 'sedan'}
    data['body_type'] = data['body'].apply(lambda x: body_dict[x])
    data.brand = data.brand.apply(lambda x: x.upper())
    data.brand = data.brand.apply(lambda x: 'MERCEDES-BENZ' if x == 'MERCEDES' else x)
    color_dict = {'040001': 'чёрный', 'FAFBFB': 'белый', '97948F': 'серый', 'CACECB': 'серебристый', '0000CC': 'синий',
              '200204': 'коричневый', 'EE1D19': 'красный', '007F00': 'зелёный', 'C49648': 'бежевый',
              '22A0F8': 'голубой', '660099': 'пурпурный', 'DEA522': 'золотистый', '4A2197': 'фиолетовый',
              'FFD600': 'жёлтый', 'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}
    data.color.replace(to_replace=color_dict, inplace=True)
    data.displacement = data.displacement.apply(lambda x: x.replace(" LTR", "0.0 LTR") if x == " LTR" else x)
    data.displacement = data.displacement.apply(lambda x: float(x.replace("LTR", "")) if isinstance(x, str) else x)
    data.power = data.power.apply(lambda x: float(x.replace("N12", "")) if isinstance(x, str) else x)

    fuel_dict={'DIESEL':'D', 'дизель':'D', 'GASOLINE':'B', 'бензин':'B', 'HYBRID':'H', 'гибрид':'H', 'ELECTRO':'E', 'электро':'E', 'LPG':'G', 'газ':'G'}
    data.fuel = data.fuel.apply(lambda x: fuel_dict[x])

    data.mileage = data.mileage.apply(lambda x: float(x))
    data.mileage = data.mileage.apply(lambda x: 10.0 if x < 10.0 else x)

    data.model_year = data.model_year.apply(lambda x: int(x))
    data.manuf_year = data.manuf_year.apply(lambda x: int(x))
    if 'price' in data.columns:
            data.price = data.price.apply(lambda x: int(x))

    data.model = data.model.apply(lambda x: x.upper())

    data.doors = data.doors.apply(lambda x: int(x))
    data.doors = data.doors.apply( lambda x: 2 if x ==0 else x)

    data.num_owners.fillna(0, inplace=True)
    data.num_owners = data.num_owners.apply( lambda x: int(x[0]) if isinstance(x, str) else int(x))
    data.num_owners = data.num_owners.apply(lambda x: 3 if x>2 else x)

    transmission_dict = {'AUTOMATIC':'AT', 'автоматическая':'AT', 'ROBOT': 'AMT', 'роботизированная':'AMT', 'MECHANICAL':'MT', 'механическая':'MT', 'VARIATOR':'CVT', 'вариатор':'CVT'}
    data.transmission = data.transmission.apply(lambda x: transmission_dict[x])

    data.car_license.fillna('ORIGINAL', inplace=True)
    license_dict={'ORIGINAL':'ORG', 'Оригинал':'ORG', 'DUPLICATE':'DPL', 'Дубликат':'DPL'}
    data.car_license = data.car_license.apply(lambda x: license_dict[x])

    gear_dict = {'REAR_DRIVE':'RWD', 'ALL_WHEEL_DRIVE':'AWD', 'FORWARD_CONTROL': 'FWD', 'задний':'RWD', 'полный':'AWD', 'передний': 'FWD'}
    data.gear = data.gear.apply(lambda x: gear_dict[x])

    steering_dict = {'LEFT':'L', 'RIGHT':'R', 'Левый':'L', 'Правый':'R'}
    data.steering = data.steering.apply(lambda x: steering_dict[x])

    data.vendor = data.vendor.apply(lambda x: 'EUROPEAN' if x == 'AMERICAN' else x)

    return data

В дальнешем функция перекочевала в модуль P05_01_Lib.py

####  Построение прототипа модели

Сначала проверю функцию первоначальной обработки данных еще раз на тех же данных.

In [132]:
df =  pd.read_csv('input/211121-12brands.csv')
df = initial_transformation(df)
for colum in cat_cols:
    df[colum] = df[colum].astype('category').cat.codes

X = df[num_cols + cat_cols]
y = df.price
X.info()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14,15,19,20,22,28,33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42045 entries, 0 to 42045
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   displacement  42045 non-null  float64
 1   power         42045 non-null  float64
 2   mileage       42045 non-null  float64
 3   manuf_year    42045 non-null  int64  
 4   brand         42045 non-null  int8   
 5   color         42045 non-null  int8   
 6   fuel          42045 non-null  int8   
 7   model         42045 non-null  int16  
 8   doors         42045 non-null  int8   
 9   transmission  42045 non-null  int8   
 10  vendor        42045 non-null  int8   
 11  num_owners    42045 non-null  int8   
 12  car_license   42045 non-null  int8   
 13  gear          42045 non-null  int8   
 14  steering      42045 non-null  int8   
 15  body_type     42045 non-null  int8   
dtypes: float64(3), int16(1), int64(1), int8(11)
memory usage: 2.1 MB


In [133]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, shuffle=True, random_state=42)
rfr_log = RandomForestRegressor(random_state=42, n_jobs=-1)
rfr_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rfr_log.predict(X_test))

print(f"The MAPE mertic for the Random Forest model on ln(price) : {(mape(y_test, predict_rf_log) * 100):0.4f}%.")

The MAPE mertic for the Random Forest model on ln(price) : 11.5815%.


Вызывает подозрения своей стабильностью кодировщик категорийных признаков. Перепишу его под фиксированные категории. Категории после присвоения номеров оказываются упорядочены.  Первый кодировщик будет брать последовательность случайно. Второй - по возрастанию частоты использования (чаще - больше номер).

In [134]:
def parse_simple_encoder(data, cols):
    parse_dict = dict()
    for col in cols:
        di = dict()
        li = list(data[col].unique())
        n = 0
        for v in li:
            di[v]=n
            n+=1
        parse_dict[col] = di
    return parse_dict

In [135]:
df = pd.read_csv('input/211121-12brands.csv')
df = initial_transformation(df)
se_dict = parse_simple_encoder(df,cat_cols)
# df.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14,15,19,20,22,28,33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [144]:
def apply_encoder(data, cols, enc_dic):
    li = []
    for c in cols:
        enc = enc_dic[c]
        vals = enc.keys()
        # print(enc)
        nc = c+'_enc'
        data[nc] = data[c].apply(lambda x: enc[x] if x in vals else 0)
        li.append(nc)
    return li, data

In [137]:
new_cat_cols, df = apply_encoder(df, cat_cols, se_dict)

In [138]:
df.head()

,Unnamed: 0,index,body,brand,url,color,displacement,power,fuel,mileage,...,fuel_enc,model_enc,doors_enc,transmission_enc,vendor_enc,num_owners_enc,car_license_enc,gear_enc,steering_enc,body_type_enc
0,0,0,внедорожник,BMW,https://auto.ru/cars/used/sale/bmw/x5/11057182...,чёрный,3.0,249.0,D,99869.0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/1106015...,чёрный,1.6,136.0,B,129000.0,...,1,1,1,0,0,1,0,1,0,1
2,2,2,седан,BMW,https://auto.ru/cars/used/sale/bmw/m5/11057329...,серебристый,4.4,600.0,B,37000.0,...,1,2,1,1,0,2,0,1,0,1
3,3,3,купе,BMW,https://auto.ru/cars/used/sale/bmw/3er/1103184...,красный,1.8,90.0,B,440000.0,...,1,1,2,2,0,1,1,1,0,2
4,4,4,седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1105925...,чёрный,4.8,367.0,B,129500.0,...,1,3,1,0,0,0,0,1,0,1


In [139]:
print(new_cat_cols)

['brand_enc', 'color_enc', 'fuel_enc', 'model_enc', 'doors_enc', 'transmission_enc', 'vendor_enc', 'num_owners_enc', 'car_license_enc', 'gear_enc', 'steering_enc', 'body_type_enc']


In [140]:
X = df[num_cols + new_cat_cols]
y = df.price
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42045 entries, 0 to 42045
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   displacement      42045 non-null  float64
 1   power             42045 non-null  float64
 2   mileage           42045 non-null  float64
 3   manuf_year        42045 non-null  int64  
 4   brand_enc         42045 non-null  int64  
 5   color_enc         42045 non-null  int64  
 6   fuel_enc          42045 non-null  int64  
 7   model_enc         42045 non-null  int64  
 8   doors_enc         42045 non-null  int64  
 9   transmission_enc  42045 non-null  int64  
 10  vendor_enc        42045 non-null  int64  
 11  num_owners_enc    42045 non-null  int64  
 12  car_license_enc   42045 non-null  int64  
 13  gear_enc          42045 non-null  int64  
 14  steering_enc      42045 non-null  int64  
 15  body_type_enc     42045 non-null  int64  
dtypes: float64(3), int64(13)
memory usage: 5

In [141]:

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, shuffle=True, random_state=42)
rfr_log = RandomForestRegressor(random_state=42, n_jobs=-1)
rfr_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rfr_log.predict(X_test))

print(f"The MAPE mertic for the Random Forest model on ln(price) : {(mape(y_test, predict_rf_log) * 100):0.4f}%.")

The MAPE mertic for the Random Forest model on ln(price) : 11.7016%.


Простая, почти случайнай кодировка столбцов ухудшила значение метрики. Попробую сделать ранжирующий кодировщик.
Идея ранжирующего кодировщика состоит в том, чтобы сначала отсортировать значения признака по количеству значений с этим признаком, а потом присваивать больший номер категории значению с большим количеством значений.

In [123]:
def parse_ranging_encoder(data, cols):
    parse_dict = dict()
    for c in cols:
        di = dict()
        li = list(data[c].value_counts(ascending=True).index)
        n = 0
        for v in li:
            di[v]=n
            n+=1
        parse_dict[c] = di
    return parse_dict

In [124]:
rg_dict = parse_ranging_encoder(df, cat_cols)
new_cat_cols, df = apply_encoder(df, cat_cols, rg_dict)

In [125]:
X = df[num_cols + new_cat_cols]
y = df.price
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, shuffle=True, random_state=42)
rfr_log = RandomForestRegressor(random_state=42, n_jobs=-1)
rfr_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rfr_log.predict(X_test))

print(f"The MAPE mertic for the Random Forest model on ln(price) : {(mape(y_test, predict_rf_log) * 100):0.4f}%.")

The MAPE mertic for the Random Forest model on ln(price) : 11.5731%.


Кодировка с помощью ранжирующего кодировщика незначительно, на 100 б.п. улучшила значение метрики. "По зернышку".


Теперь проверка наивной модели на большом наборе.

In [126]:
df36 =  pd.read_csv('input/211121-36brands.csv')
df36 = initial_transformation(df36)
rg_dict36 = parse_ranging_encoder(df36,cat_cols)
new_cat_cols, df36 = apply_encoder(df36, cat_cols, rg_dict36)
X36 = df36[num_cols + new_cat_cols]
y36 = df36.price
X36.info()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15,16,20,21,23,29,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 90999 entries, 0 to 90999
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   displacement      90999 non-null  float64
 1   power             90999 non-null  float64
 2   mileage           90999 non-null  float64
 3   manuf_year        90999 non-null  int64  
 4   brand_enc         90999 non-null  int64  
 5   color_enc         90999 non-null  int64  
 6   fuel_enc          90999 non-null  int64  
 7   model_enc         90999 non-null  int64  
 8   doors_enc         90999 non-null  int64  
 9   transmission_enc  90999 non-null  int64  
 10  vendor_enc        90999 non-null  int64  
 11  num_owners_enc    90999 non-null  int64  
 12  car_license_enc   90999 non-null  int64  
 13  gear_enc          90999 non-null  int64  
 14  steering_enc      90999 non-null  int64  
 15  body_type_enc     90999 non-null  int64  
dtypes: float64(3), int64(13)
memory usage: 1

In [127]:
X_train36, X_test36, y_train36, y_test36 = train_test_split( X36, y36, test_size=0.2, shuffle=True, random_state=42)
rfr_log36 = RandomForestRegressor(random_state=42, n_jobs=-1)
rfr_log36.fit(X_train36, np.log(y_train36))
predict_rf_log36 = np.exp(rfr_log36.predict(X_test36))
predict_rf_log12_36 = np.exp(rfr_log36.predict(X_test))
print(f"The MAPE mertic for the Random Forest model on ln(price) with 36 brands: {(mape(y_test36, predict_rf_log36) * 100):0.4f}%.")
print(f"The MAPE mertic for the Random Forest model on ln(price), trained on 36 brands, and predicting on 12 brands : {(mape(y_test, predict_rf_log12_36) * 100):0.4f}%.")


The MAPE mertic for the Random Forest model on ln(price) with 36 brands: 13.1359%.
The MAPE mertic for the Random Forest model on ln(price), trained on 36 brands, and predicting on 12 brands : 32.5796%.


Первоначальное значение МАРЕ оказалось хуже - 12.5316%, но я сравнивал на отложенной части большего объема.
Предпринята попытка обучить на 36 брендах и затем проверить на 12. Результат оказался еще хуже.
Ситуация естественная. В случае данных по 36 брендам имеем в среднем 2.5к записей на бренд.  В случае данных по 12 брендам получается 3.5к записей на бренд, т.е. при обучении на сфокусированном наборе результат получается лучше.  ((

In [142]:
for c in cat_cols:
    print(c, rg_dict[c])

brand {'INFINITI': 0, 'HONDA': 1, 'LEXUS': 2, 'VOLVO': 3, 'MITSUBISHI': 4, 'VOLKSWAGEN': 5, 'TOYOTA': 6, 'AUDI': 7, 'BMW': 8, 'NISSAN': 9, 'SKODA': 10, 'MERCEDES-BENZ': 11}
color {'розовый': 0, 'фиолетовый': 1, 'жёлтый': 2, 'золотистый': 3, 'пурпурный': 4, 'оранжевый': 5, 'голубой': 6, 'бежевый': 7, 'зелёный': 8, 'красный': 9, 'коричневый': 10, 'серебристый': 11, 'синий': 12, 'серый': 13, 'белый': 14, 'чёрный': 15}
fuel {'G': 0, 'E': 1, 'H': 2, 'D': 3, 'B': 4}
model {'AVENSIS_VERSO': 0, 'PRIUSPLUS': 1, 'PIXIS_EPOCH': 2, 'CELSIOR': 3, 'MR_S': 4, 'CAVALIER': 5, 'PASEO': 6, 'OPA': 7, 'CARINA_ED': 8, 'RNESSA': 9, 'TACOMA': 10, 'IST': 11, 'CORONA_EXIV': 12, 'PIXO': 13, 'PLATZ': 14, 'VEROSSA': 15, 'TOURING_HIACE': 16, 'SCEPTER_SEDAN': 17, 'SPARKY': 18, 'SPRINTER_TRUENO': 19, 'MASTER_ACE_SURF': 20, 'AYGO': 21, 'LAFESTA': 22, 'ROOMY': 23, 'POLO_GTI': 24, 'ALLEX': 25, 'CRESSIDA': 26, 'ARMADA': 27, 'BLUEBIRD_SYLPHY': 28, 'REGIUSACE': 29, 'PULSAR': 30, 'EXPERT': 31, 'QUEST': 32, 'CLC_KLASSE': 33,

In [145]:
tdf =  pd.read_csv('input/test.csv')
tdf = initial_transformation(tdf)
# rg_dict36 = parse_ranging_encoder(df36,cat_cols)  - словарь использую со времени обучения
new_cat_cols, tdf = apply_encoder(tdf, cat_cols, rg_dict)
Xt = tdf[num_cols + new_cat_cols]
Xt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   displacement      34686 non-null  float64
 1   power             34686 non-null  float64
 2   mileage           34686 non-null  float64
 3   manuf_year        34686 non-null  int64  
 4   brand_enc         34686 non-null  int64  
 5   color_enc         34686 non-null  int64  
 6   fuel_enc          34686 non-null  int64  
 7   model_enc         34686 non-null  int64  
 8   doors_enc         34686 non-null  int64  
 9   transmission_enc  34686 non-null  int64  
 10  vendor_enc        34686 non-null  int64  
 11  num_owners_enc    34686 non-null  int64  
 12  car_license_enc   34686 non-null  int64  
 13  gear_enc          34686 non-null  int64  
 14  steering_enc      34686 non-null  int64  
 15  body_type_enc     34686 non-null  int64  
dtypes: float64(3), int64(13)
memory usage: 4

In [146]:
tdf.columns

Index(['body', 'brand', 'url', 'color', 'displacement', 'power', 'fuel',
       'mileage', 'model_year', 'model', 'doors', 'manuf_year', 'sell_id',
       'transmission', 'vendor', 'num_owners', 'car_license', 'gear',
       'steering', 'body_type', 'brand_enc', 'color_enc', 'fuel_enc',
       'model_enc', 'doors_enc', 'transmission_enc', 'vendor_enc',
       'num_owners_enc', 'car_license_enc', 'gear_enc', 'steering_enc',
       'body_type_enc'],
      dtype='object')

In [148]:
tdf['price'] = np.exp(rfr_log.predict(Xt))
subm = tdf[['sell_id','price']]
subm.head(10)

,sell_id,price
0,1100575026,1.102437e+06
1,1100549428,1.199589e+06
2,1100658222,1.881145e+06
3,1100937408,1.059686e+06
4,1101037972,1.126483e+06
5,1100912634,1.284442e+06
6,1101228730,9.849121e+05
7,1100165896,6.332180e+05
8,1100768262,2.680414e+06
9,1101218501,1.113166e+06


In [149]:
subm.to_csv(f'211123_subm_MVP.csv', index=False)

Счет на Kaggle  - 53.46337.  Разочаровывает.

### Промежуточные выводы

Прототип для оценивания сделан. Необходимо скорректировать данные на инфляцию и создать новые признаки, в том числе с помощью данных, полученных из паутины.

Принято решение отказаться от данных по 36 брендам (возможно ошибочно) и сосредоточиться на построении и обучении модели по 12 брендам.